In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np
import json
import math
import os
import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import ResNet50,MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile,VGG16
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard,CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
import gc
from functools import partial
from sklearn import metrics
from collections import Counter
import json


In [ ]:
#Transfer 'jpg' images to an array IMG
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".jpg":
            img = read(PATH)

            img = cv2.resize(img, (RESIZE,RESIZE))

            IMG.append(np.array(img))
    return IMG

data_path = r"C:\Users\aliil\OneDrive\Masaüstü\x5"

benign_test = np.array(Dataset_loader(data_path+'/test/Healthy',224))
malign_test = np.array(Dataset_loader(data_path+'/test/Infected',224))

In [ ]:
# Create labels
benign_test_label = np.zeros(len(benign_test))
malign_test_label = np.ones(len(malign_test))

# Merge data 
X_test = np.concatenate((benign_test, malign_test), axis = 0)
Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

# Shuffle test data
s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

Y_test = to_categorical(Y_test, num_classes= 2)

In [ ]:
import tensorflow.keras
import tensorflow as tf
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
keras_model = tensorflow.keras.models.load_model(r'C:\Users\aliil\OneDrive\Masaüstü\hdf5\Resnet50.x3.weights.best.hdf5', compile=False)
keras_model._name = 'model1'
keras_model2 = tensorflow.keras.models.load_model(r'C:\Users\aliil\OneDrive\Masaüstü\hdf5\vgg16.x3.weights.best.hdf5', compile=False)
keras_model2._name = 'model2'
keras_model3 = tensorflow.keras.models.load_model(r'C:\Users\aliil\OneDrive\Masaüstü\hdf5\vgg16.x3.weights.best.hdf5', compile=False)
keras_model3._name = 'model3'
models = [keras_model, keras_model2, keras_model3]
model_input = tf.keras.Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)  

ensemble_model.save("ensemblemodel.h5")
print()

In [ ]:
ensemble_model.load_weights("ensemble.h5")

In [ ]:
Y_val_pred = ensemble_model.predict(X_test)

In [ ]:
Y_pred = ensemble_model.predict(X_test)

In [ ]:
tta_steps = 10
predictions = []
for i in tqdm(range(tta_steps)):
    preds = ensemble_model.predict(X_test,
                                    steps = 1)#len(X_test)/BATCH_SIZE)
    
    predictions.append(preds)
    gc.collect()
    
Y_pred_tta = np.mean(predictions, axis=0)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report( np.argmax(Y_test, axis=1), np.argmax(Y_pred_tta, axis=1), digits=4))

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=55)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
  

cm = confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(Y_pred, axis=1))

cm_plot_label =['healthy', 'infected']
plot_confusion_matrix(cm, cm_plot_label, title ='Confusion Metrix for ALL')

In [ ]:
i=0
prop_class=[]
mis_class=[]

for i in range(len(Y_test)):
        prop_class.append(i)
    

i=0
for i in range(len(Y_test)):
        mis_class.append(i)
   

# # Display first 8 images of benign
w=224
h=224
fig=plt.figure(figsize=(14, 30))
columns = 5
rows = 9

def Transfername(namecode):
    if namecode==0:
        return "Healty"
    else:
        return "Infected"
    
for i in range(len(prop_class)):
    ax = fig.add_subplot(rows, columns, i+1)
    ax.set_title("Predicted result:"+ Transfername(np.argmax(Y_pred_tta[prop_class[i]]))
                       +"\n"+"Actual result: "+ Transfername(np.argmax(Y_test[prop_class[i]])))
                      
    plt.imshow(X_test[prop_class[i]], interpolation='nearest')
plt.show()
